In [8]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
import pickle

In [9]:
train_path = "./feature.train"
valid_path = "./feature.valid"
train_valid_path = "./feature.train_valid"
test_path = "./feature.test"

### Create training set (train + valid)

In [34]:
X_train = []
y_train = []

with open(train_valid_path, 'r') as f:
    for line in f:
        items = line.split(" %%%% ")
        lineid = items[0].split("-")[0] + str(int(items[0].split("-")[1])+1)
        x = np.fromstring(items[1], sep=' ')

        X_train.append(x)
        y_train.append(items[2].strip())

print("done")

done


In [11]:
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape)
print(y_train.shape)

(86755, 600)
(86755,)


### Create test set

In [10]:
X_test = []
y_test = []

with open(test_path, 'r') as f:
    for line in f:
        items = line.split(" %%%% ")
        lineid = items[0].split("-")[0] + str(int(items[0].split("-")[1])+1)
        x = np.fromstring(items[1], sep=' ')

        X_test.append(x)
        y_test.append(items[2].strip())
        
print("done")

done


In [11]:
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_test.shape)
print(y_test.shape)

(21687, 600)
(21687,)


### Try on smaller dataset first

In [12]:
X_train_small = X_train[:1000]
y_train_small = y_train[:1000]

print(X_train_small.shape)
print(y_train_small.shape)

(1000, 600)
(1000,)


### Run logistic regression - fit the model on (X_train, y_train)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
print("model trained!")

In [ ]:
pickle.dump( clf, open( "lr_fit_train_valid.pkl", "wb" ) )

In [14]:
predicted_train = clf.predict(X_train)
accuracy_on_train = 100.0 * np.mean(predicted_train == y_train)
print("accuracy_on_train: {}".format(accuracy_on_train))

accuracy_on_train: 52.7


### Load the model from pickle

In [12]:
clf = pickle.load( open( "lr_fit_train_valid.pkl", "rb" ) )

### Test the LR model

In [13]:
predicted = clf.predict(X_test)
accuracy = 100.0 * np.mean(predicted == y_test)
print("accuracy on test dataset: {}".format(accuracy))

accuracy on test dataset: 71.34688984184073


### Get the top-n relation predictions

In [38]:
probs = clf.predict_proba(X_test)
n = 5
best_n = np.argsort(probs, axis=1)[:,-5:]
best_prob = np.sort(probs, axis=1)[:,-5:]
best_rels = clf.classes_[best_n]
best_rels.shape

(21687, 5)

In [45]:
probs.shape

(21687, 1698)

In [39]:
y_test[34]

'www.freebase.com/people/person/place_of_birth'

In [41]:
outfile = open("topk-retrieval-test-hits-5.txt", "w")
for i, (top_rels, top_probs) in enumerate(zip(best_rels, best_prob)):
    gold = y_test[i]
    for rel, prob in zip(top_rels, top_probs):
        if gold == rel:
            label = 1
        else:
            label = 0
        outfile.write("{}-{} %%%% {} %%%% {} %%%% {}\n".format("test", i+1, rel, label, prob))
print("done!")

done!
